# **LP2_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [8]:
problem = """You are in charge of the supply purchasing of a company that produces two kinds of drugs.
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market.

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales.

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded.
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available.
Your company can also only store a limited amount of raw materials.
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces.
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively.
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. """

In [9]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [10]:
print(response1.choices[0].message.content)

1. D1, D2: Quantity of drugs 1 and 2 to be produced.
2. RM1, RM2: Quantity of raw materials 1 and 2 to be purchased.
3. A1, A2: Quantity of active agents extracted from raw materials 1 and 2, these are to be determined by given extraction rates which can vary.
4. P_D1, P_D2: Profit produced from drug 1 and 2.
5. C_RM1, C_RM2: Costs of raw materials 1 and 2.
6. C_OP1, C_OP2: Operational costs for producing drug 1 and 2.
7. B: Given budget for purchasing raw materials and operating production.
8. M: Manpower required for the production of the drugs.
9. E: Equipment hours required for the production of the drugs.
10. S_RM: Maximum storage capacity of raw materials.
11. A_D1, A_D2: The required amount of active ingredient in drugs 1 and 2.


In [11]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [12]:
print(response2.choices[0].message.content)

Maximize Z = P_D1*D1 + P_D2*D2 - C_RM1*RM1 - C_RM2*RM2 - C_OP1*D1 - C_OP2*D2


In [13]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [14]:
print(response3.choices[0].message.content)

1. C_RM1 * RM1 + C_RM2 * RM2 + C_OP1 * D1 + C_OP2 * D2 ≤ B
2. M1 * D1 + M2 * D2 ≤ M
3. E1 * D1 + E2 * D2 ≤ E
4. RM1 ≤ S_RM
5. RM2 ≤ S_RM
6. A1 ≥ A_D1 
7. A2 ≥ A_D2
8. A1 ≤ RM1 * 1.005
9. A2 ≤ RM2 * 1.02
10. A1 ≥ RM1 * 0.995
11. A2 ≥ RM2 * 0.98
12. D1, D2, RM1, RM2 ≥ 0


### **Generate Pyomo Code**

In [15]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [17]:
Markdown(response4.choices[0].message.content)

Here is a Python pyomo code for the given optimization problem:

```python

from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define the decision variables
model.D1 = Var(domain=NonNegativeReals)
model.D2 = Var(domain=NonNegativeReals)
model.RM1 = Var(domain=NonNegativeReals)
model.RM2 = Var(domain=NonNegativeReals)
model.A1 = Var(domain=NonNegativeReals)
model.A2 = Var(domain=NonNegativeReals)

# Here we are using random data, Please replace it with your actual data
P_D1 = 10
P_D2 = 20
C_RM1 = 3
C_RM2 = 2
C_OP1 = 4
C_OP2 = 2

# Create a objective function
model.profit = Objective(expr = P_D1*model.D1 + P_D2*model.D2 - C_RM1*model.RM1 - C_RM2*model.RM2 - C_OP1*model.D1 - C_OP2*model.D2, sense=maximize)

# Construct constraints
B = 100 # Budget
M = 10 # Manpower
E = 20 # Equipment
S_RM = 25 # Storage
A_D1 = 3 # Active ingredient in Drug1
A_D2 = 2 # Active ingredient in Drug2 

# Create the constraints

model.constraint1 = Constraint(expr = C_RM1*model.RM1 + C_RM2*model.RM2 + C_OP1*model.D1 + C_OP2*model.D2 <= B)

# Assuming M1=M2 and E1=E2 are some constants which indicate manpower and equipment hours
M1 = 1
M2 = 1
E1 = 1
E2 = 1
model.constraint2 = Constraint(expr = M1*model.D1 + M2*model.D2 <= M)
model.constraint3 = Constraint(expr = E1*model.D1 + E2*model.D2 <= E)

model.constraint4 = Constraint(expr = model.RM1 <= S_RM)
model.constraint5 = Constraint(expr = model.RM2 <= S_RM)

model.constraint6 = Constraint(expr = model.A1 >= A_D1)
model.constraint7 = Constraint(expr = model.A2 >= A_D2)

model.constraint8 = Constraint(expr = model.A1 <= model.RM1*(1.005))
model.constraint9 = Constraint(expr = model.A2 <= model.RM2*(1.02))

model.constraint10 = Constraint(expr = model.A1 >= model.RM1*(0.995))
model.constraint11 = Constraint(expr = model.A2 >= model.RM2*(0.98))

#Display output    
SolverFactory('glpk').solve(model)
model.pprint()
    
```
You replace the sample data with your actual data and run the model. This model maximizes the profit by considering the costs and constraints related to raw materials, active agents, and the production process.

### **Run the code Generated by GPT4**

In [19]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define the decision variables
model.D1 = Var(domain=NonNegativeReals)
model.D2 = Var(domain=NonNegativeReals)
model.RM1 = Var(domain=NonNegativeReals)
model.RM2 = Var(domain=NonNegativeReals)
model.A1 = Var(domain=NonNegativeReals)
model.A2 = Var(domain=NonNegativeReals)

# Here we are using random data, Please replace it with your actual data
P_D1 = 10
P_D2 = 20
C_RM1 = 3
C_RM2 = 2
C_OP1 = 4
C_OP2 = 2

# Create a objective function
model.profit = Objective(expr = P_D1*model.D1 + P_D2*model.D2 - C_RM1*model.RM1 - C_RM2*model.RM2 - C_OP1*model.D1 - C_OP2*model.D2, sense=maximize)

# Construct constraints
B = 100 # Budget
M = 10 # Manpower
E = 20 # Equipment
S_RM = 25 # Storage
A_D1 = 3 # Active ingredient in Drug1
A_D2 = 2 # Active ingredient in Drug2

# Create the constraints

model.constraint1 = Constraint(expr = C_RM1*model.RM1 + C_RM2*model.RM2 + C_OP1*model.D1 + C_OP2*model.D2 <= B)

# Assuming M1=M2 and E1=E2 are some constants which indicate manpower and equipment hours
M1 = 1
M2 = 1
E1 = 1
E2 = 1
model.constraint2 = Constraint(expr = M1*model.D1 + M2*model.D2 <= M)
model.constraint3 = Constraint(expr = E1*model.D1 + E2*model.D2 <= E)

model.constraint4 = Constraint(expr = model.RM1 <= S_RM)
model.constraint5 = Constraint(expr = model.RM2 <= S_RM)

model.constraint6 = Constraint(expr = model.A1 >= A_D1)
model.constraint7 = Constraint(expr = model.A2 >= A_D2)

model.constraint8 = Constraint(expr = model.A1 <= model.RM1*(1.005))
model.constraint9 = Constraint(expr = model.A2 <= model.RM2*(1.02))

model.constraint10 = Constraint(expr = model.A1 >= model.RM1*(0.995))
model.constraint11 = Constraint(expr = model.A2 >= model.RM2*(0.98))

# Display output
SolverFactory('glpk').solve(model)
model.pprint()

6 Var Declarations
    A1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   3.0 :  None : False : False : NonNegativeReals
    A2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   2.0 :  None : False : False : NonNegativeReals
    D1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    D2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  10.0 :  None : False : False : NonNegativeReals
    RM1 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 2.98507462686567 :  None : False : False : NonNegativeReals
    RM2 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 1.96078431372549 :  Non

In [21]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define the decision variables
model.D1 = Var(domain=NonNegativeReals)
model.D2 = Var(domain=NonNegativeReals)
model.RM1 = Var(domain=NonNegativeReals)
model.RM2 = Var(domain=NonNegativeReals)
model.A1 = Var(domain=NonNegativeReals)
model.A2 = Var(domain=NonNegativeReals)

# Here we are using random data, Please replace it with your actual data
P_D1 = 6200
P_D2 = 6900
C_RM1 = 100
C_RM2 = 199.90
C_OP1 = 700
C_OP2 = 800

# Create a objective function
model.profit = Objective(expr = P_D1*model.D1 + P_D2*model.D2 - C_RM1*model.RM1 - C_RM2*model.RM2 - C_OP1*model.D1 - C_OP2*model.D2, sense=maximize)

# Construct constraints
B = 100000 # Budget
M = 2000 # Manpower
E = 800 # Equipment
S_RM = 1000 # Storage
A_D1 = 0.5 # Active ingredient in Drug1
A_D2 = 0.6 # Active ingredient in Drug2

# Create the constraints

model.constraint1 = Constraint(expr = C_RM1*model.RM1 + C_RM2*model.RM2 + C_OP1*model.D1 + C_OP2*model.D2 <= B)

# Assuming M1=M2 and E1=E2 are some constants which indicate manpower and equipment hours
M1 = 90
M2 = 100
E1 = 40
E2 = 50
model.constraint2 = Constraint(expr = M1*model.D1 + M2*model.D2 <= M)
model.constraint3 = Constraint(expr = E1*model.D1 + E2*model.D2 <= E)

model.constraint4 = Constraint(expr = model.RM1 <= S_RM)
model.constraint5 = Constraint(expr = model.RM2 <= S_RM)

model.constraint6 = Constraint(expr = model.A1 >= A_D1)
model.constraint7 = Constraint(expr = model.A2 >= A_D2)

model.constraint8 = Constraint(expr = model.A1 <= model.RM1*(1.005))
model.constraint9 = Constraint(expr = model.A2 <= model.RM2*(1.02))

model.constraint10 = Constraint(expr = model.A1 >= model.RM1*(0.995))
model.constraint11 = Constraint(expr = model.A2 >= model.RM2*(0.98))

# Display output
SolverFactory('glpk').solve(model)
print(model.profit())

109832.6605209248


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**